In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial import cKDTree

# Load YOLO model
model=YOLO("yolo11n.pt")

def track(s):
    # Initialize video capture and background
    video=cv2.VideoCapture(s)
    ret, background=video.read()
    if ret:
        background=cv2.resize(background, (640, 384))

    # Tracking parameters
    movement={}  # Dictionary to store paths for each person
    next_id=1  # Unique ID for each person
    max_distance=50  # Maximum distance to match detections to existing tracks
    
    while True:
        ret, frame=video.read()
        if not ret:
            break

        frame=cv2.resize(frame, (640, 384))
        results=model(frame)
        detected_people=[]  # Store current frame's detections

        # Parse the detection results
        for result in results:
            for box in result.boxes:
                if int(box.cls[0]) == 0:  # 0 is the class ID for 'person'
                    x1, y1, x2, y2=map(int, box.xyxy[0])  # Extract bounding box
                    feet_x=int((x1+x2)/2)  # Calculate feet position
                    feet_y=y2
                    detected_people.append((feet_x, feet_y))

        # Match detections to existing tracks
        updated_movement={}
        for person_id, path in movement.items():
            last_position=path[-1]  # Last recorded position of the person
            matched=False

            # Match current detections to the person's last position
            for idx, (feet_x, feet_y) in enumerate(detected_people):
                distance=np.linalg.norm(np.array(last_position)-np.array((feet_x, feet_y)))
                if distance<max_distance:  # Match found
                    updated_movement[person_id]=path+[(feet_x, feet_y)]
                    detected_people.pop(idx)  # Remove matched detection
                    matched=True
                    break

            # If no match, keep the path as is
            if not matched:
                updated_movement[person_id]=path

        # Assign new IDs to unmatched detections
        for feet_x, feet_y in detected_people:
            updated_movement[next_id]=[(feet_x, feet_y)]
            next_id+=1

        movement=updated_movement

    video.release()

    # Convert the background to RGB for matplotlib
    background_rgb=cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
    return background_rgb, movement

def printPath(a, b):
    # Plot the paths on the background image
    plt.figure(figsize=(10, 6))
    plt.imshow(a)

    # Plot each person's path with a unique color
    for person_id, path in b.items():
        path=np.array(path)
        if len(path)>0:
            plt.plot(path[:, 0], path[:, 1], label=f'Person {person_id}')

    plt.legend()
    plt.title("Paths of Detected People")
    plt.show()

def printSingle(a, b):
    plt.figure(figsize=(10, 6))
    plt.imshow(a)
    plt.plot(b[:, 0], b[:, 1], 'r-')
    plt.show()

background_rgb1, movement1=track('Feed/Feed1.mp4')
mov1=np.array(movement1[1])
printPath(background_rgb1, movement1)
mov1=np.delete(mov1, [0,1,2,3,4,5,78], axis=0)
printSingle(background_rgb1, mov1)
np.save('movement_1.npy', mov1)

background_rgb2, movement2=track('Feed/Feed2.mp4')
mov2=np.array(movement2[1])
printPath(background_rgb2, movement2)
mov2=np.delete(mov2, [0], axis=0)
printSingle(background_rgb2, mov2)
np.save('movement_2.npy', mov2)

def derive_transformation_matrix(points, mapped_points):
    A=[]
    for i in range(len(points)):
        xi, yi=points[i]
        ui, vi=mapped_points[i]
        A.append([xi, yi, 1, 0, 0, 0, -ui*xi, -ui*yi, -ui])
        A.append([0, 0, 0, xi, yi, 1, -vi*xi, -vi*yi, -vi])
    A=np.array(A)
    _, _, V=np.linalg.svd(A)
    H=V[-1]/V[-1, -1]
    H_matrix=H.reshape((3, 3))
    return H_matrix

def apply_transformation(matrix, x, y):
    point=np.array([x, y, 1])
    transformed_point=np.dot(matrix, point)
    u=transformed_point[0]/transformed_point[2]
    v=transformed_point[1]/transformed_point[2]
    return round(u, 8), round(v, 8)

arr=np.load('movement_1.npy')

fig=plt.figure(figsize=(10, 6))
plt.plot(arr[:, 0], arr[:, 1], 'r-', markersize=1, linewidth=1)
plt.plot([0, 0], [215, 384], 'bo-', linewidth=2)
plt.plot([0, 640], [384, 384], 'bo-', linewidth=2)
plt.plot([640, 640], [384, 200], 'bo-', linewidth=2)
plt.plot([640, 0], [200, 215], 'bo-', linewidth=2)
plt.xlim(0, 640)
plt.ylim(384, 0)
plt.show()

points=[(0, 215), (0, 384), (640, 384), (640, 200)]
mapped_points=[(0, 0), (0, 384), (640, 384), (640, 0)]
transformation_matrix=derive_transformation_matrix(points, mapped_points)

newarr=[]
for i in arr:
    point=np.dot(transformation_matrix, [i[0], i[1], 1])
    point/=point[2]
    u, v=round(point[0], 8), round(point[1], 8)
    newarr.append([u, v])

# Create a blank figure (without the background)
fig=plt.figure(figsize=(10, 6))
newarr=np.array(newarr)
plt.plot(newarr[:, 0], newarr[:, 1], 'r-', markersize=1, linewidth=1)
plt.plot([0, 0], [0, 384], 'bo-', linewidth=2)
plt.plot([0, 640], [384, 384], 'bo-', linewidth=2)
plt.plot([640, 640], [384, 0], 'bo-', linewidth=2)
plt.plot([640, 0], [0, 0], 'bo-', linewidth=2)
plt.xlim(0, 640)
plt.ylim(384, 0)
plt.show()

np.save('topview_1.npy', newarr)

arr2=np.load('movement_2.npy')

fig=plt.figure(figsize=(10, 6))
plt.plot(arr2[:, 0], arr2[:, 1], 'r-', markersize=1, linewidth=1)
plt.plot([0, 640], [384, 384], 'bo-', linewidth=2)
plt.plot([640, 640], [384, 140], 'bo-', linewidth=2)
plt.plot([640, 0], [140, 200], 'bo-', linewidth=2)
plt.plot([0, 0], [200, 384], 'bo-', linewidth=2)
plt.xlim(0, 640)
plt.ylim(384, 0)
plt.show()

points2=[(640, 384), (640, 140), (0, 200), (0, 384)]
mapped_points2=[(0, 0), (0, 384), (640, 384), (640, 0)]
transformation_matrix2=derive_transformation_matrix(points2, mapped_points2)

newarr2=[]
for i in arr2:
    point=np.dot(transformation_matrix2, [i[0], i[1], 1])
    point/=point[2]
    u, v=round(point[0], 8), round(point[1], 8)
    newarr2.append([u, v])

# Create a blank figure (without the background)
fig=plt.figure(figsize=(10, 6))
newarr2=np.array(newarr2)
plt.plot(newarr2[:, 0], newarr2[:, 1], 'r-', markersize=1, linewidth=1)
plt.plot([0, 0], [0, 384], 'bo-', linewidth=2)
plt.plot([0, 640], [384, 384], 'bo-', linewidth=2)
plt.plot([640, 640], [384, 0], 'bo-', linewidth=2)
plt.plot([640, 0], [0, 0], 'bo-', linewidth=2)
plt.xlim(0, 640)
plt.ylim(384, 0)
plt.show()

np.save('topview_2.npy', newarr2)

a=np.load("topview_1.npy")
b=np.load("topview_2.npy")

def compare(a, b):
    n=min(len(a), len(b))
    maxlen=0
    num=-1
    for i in range(1, n+1, 10):
        end_shape=a[-i:]
        start_shape=b[:i]
        x, y=end_shape[0]
        w, z=start_shape[0]
        shift_x=x-w
        shift_y=y-z
        start_shape_transformed=start_shape+np.array([shift_x, shift_y])
        # Compute Euclidean distances
        distances=np.linalg.norm(end_shape-start_shape_transformed, axis=1)
        threshold=5
        if np.all(distances<=threshold):
            print(f"The end of array1 matches the start of array2 within a threshold of {threshold}.")
            if(i>maxlen):
                maxlen=i
                num=i
        else:
            print(f"The shapes do not match within the threshold of {threshold}.")

        # Print distances for debugging
        print("Distances:", distances)
    print("The value of num is: ", num)
    b=b[num:]
    return a, b

a, b=compare(a, b)

# Get the last point of a and the first point of b
x, y=a[-1]
w, z=b[0]

# Calculate the coordinate transformation (shift)
shift_x=x-w
shift_y=y-z

# Apply the transformation to all points in b
b_transformed=b+np.array([shift_x, shift_y])

# Plot the arrays
plt.figure(figsize=(12, 8))
plt.plot(a[:, 0], a[:, 1], color='green', label='a (Original)')
plt.plot(b_transformed[:, 0], b_transformed[:, 1], color='red', label='b (Transformed)')
plt.xlim(-640, 1280)
plt.ylim(-384, 768)
plt.legend()
plt.title('Coordinate Transformation with Conditional Shape Matching Overlap Handling')
plt.xlabel('x-coordinate (-640 to 1280)')
plt.ylabel('y-coordinate (-384 to 768)')
plt.gca().invert_yaxis()  # Invert y-axis for portrait orientation
plt.grid(True)
plt.show()

# Print transformation details
print(f"Transformation applied: shift_x={shift_x}, shift_y={shift_y}")

def rotate_points(points, angle_deg, origin):
    angle_rad=np.deg2rad(angle_deg)
    cos_theta=np.cos(angle_rad)
    sin_theta=np.sin(angle_rad)
    
    # Rotation matrix
    rotation_matrix=np.array([
        [cos_theta, -sin_theta],
        [sin_theta, cos_theta]
    ])
    
    # Translate points to the origin (a, b)
    translated_points=points-origin
    rotated_points=np.dot(translated_points, rotation_matrix.T)
    final_points=rotated_points+origin
    
    return final_points

# Rotate all points in 'a' by 60 degrees around the point (500, 500)
rotated_b=rotate_points(b, angle_deg=25, origin=np.array([611.15345205, 212.88844794]))

# Get the last point of a and the first point of b
x, y=a[-1]
w, z=rotated_b[0]

# Calculate the coordinate transformation (shift)
shift_x=x-w
shift_y=y-z

# Apply the transformation to all points in b
rotated_b=rotated_b+np.array([shift_x, shift_y])

# Plot the original and rotated arrays
plt.figure(figsize=(12, 8))
plt.plot(a[:, 0], a[:, 1], 'green', label='Original Points')
plt.plot(rotated_b[:, 0], rotated_b[:, 1], 'red', label='Rotated Points (25°)')
plt.xlim(-640, 1280)
plt.ylim(-384, 768)
plt.legend()
plt.title('Rotation of Points by 20° Around (611.15345205, 212.88844794)')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.gca().invert_yaxis()  # Invert y-axis for portrait orientation
plt.grid(True)
plt.show()

rotate_1=rotate_points(a, angle_deg=15, origin=np.array([0,200]))
rotate_2=rotate_points(rotated_b, angle_deg=15, origin=np.array([0,200]))

# Plot the original and rotated arrays
plt.figure(figsize=(12,8))
plt.plot(rotate_1[:, 0], rotate_1[:, 1], 'green', label='Original Points')
plt.plot(rotate_2[:, 0], rotate_2[:, 1], 'red', label='Rotated Points (60°)')
plt.xlim(-640, 1280)
plt.ylim(-384, 1000)
plt.legend()
plt.title('Rotation of Points by 10° Around')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.gca().invert_yaxis()  # Invert y-axis for portrait orientation
plt.grid(True)
plt.show()

# Plot the original and rotated arrays
plt.figure(figsize=(12,8))
plt.plot(rotate_1[:, 0], rotate_1[:, 1], 'r')
plt.plot(rotate_2[:, 0], rotate_2[:, 1], 'r')
plt.xlim(-100, 1000)
plt.ylim(0, 1000)
plt.legend()
plt.title('Rotation of Points by 10° Around')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.gca().invert_yaxis()
plt.show()

arr_combined=np.concatenate((rotate_1, rotate_2), axis=0)  # Row-wise
np.save('test_points.npy', arr_combined)

background_rgb_ver, movement2_ver=track('Feed/TopFeed.mp4')
mov_ver=np.array(movement2_ver[2])
r=np.arange(-20, 0)
x=np.delete(mov_ver, r, axis=0)
printPath(background_rgb_ver, movement2_ver)
printSingle(background_rgb_ver, x)

# Create a blank figure (without the background)
fig=plt.figure(figsize=(10, 6))
x=np.array(x)
plt.plot(x[:, 0], x[:, 1], 'r-', markersize=1, linewidth=1)
plt.xlim(0, 640)
plt.ylim(384, 0)
plt.show()

np.save('check_points.npy', x)

test=np.load("test_points.npy")
check=np.load("check_points.npy")
x, y=test[0]
test=test-np.array([x, y])
w, z=check[0]
check=check-np.array([w, z])

# Plot the original and rotated arrays
plt.figure(figsize=(12,12))
plt.plot(test[:, 0], test[:, 1], 'r')
plt.plot(check[:, 0], check[:, 1], 'b')
plt.xlim(-100, 1000)
plt.ylim(-100, 1000)
plt.legend()
plt.title('Test Points')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.gca().invert_yaxis()
plt.grid()
plt.show()

# Scaling is done to get the end points same
# Compute scaling factors
scale_x=np.max(test[:, 0])/np.max(check[:, 0])
scale_y=np.max(test[:, 1])/np.max(check[:, 1])

# Apply scaling transformation
check=check*np.array([scale_x, scale_y])
print("Scaling factors:", scale_x, scale_y)

# Plot the original and rotated arrays
plt.figure(figsize=(12,12))
plt.plot(test[:, 0], test[:, 1], 'r', label="Test Plot")
plt.plot(check[:, 0], check[:, 1], 'g', label="Scaled Check Plot")
plt.xlim(-100, 1000)
plt.ylim(-100, 1000)
plt.legend()
plt.title('Comparison between the two plots')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.gca().invert_yaxis()
plt.grid()
plt.show()

# 1. Haussdorf Similarity
def hausdorff_similarity(a, b):
    d1=directed_hausdorff(a, b)[0]  # Distance A → B
    d2=directed_hausdorff(b, a)[0]  # Distance B → A
    hausdorff_dist=max(d1, d2)      # Maximum of both
    max_possible_dist=np.linalg.norm(np.max(a, axis=0)-np.min(a, axis=0))
    return 100*(1-hausdorff_dist/max_possible_dist)

# 2. Chamfer Similarity
def chamfer_similarity(a, b):
    tree_a=cKDTree(a)
    tree_b=cKDTree(b)
    dist_a_to_b, _=tree_a.query(b)  # Closest distances from B to A
    dist_b_to_a, _=tree_b.query(a)  # Closest distances from A to B
    chamfer_dist=np.mean(dist_a_to_b) + np.mean(dist_b_to_a)
    max_possible_dist=np.linalg.norm(np.max(a, axis=0)-np.min(a, axis=0))
    return 100*(1-chamfer_dist/max_possible_dist)

# Measure the similarity among the two shapes
hausdorff=hausdorff_similarity(test, check)
chamfer=chamfer_similarity(test, check)
print(f"Shape Similarity (Hausdorff-based): {hausdorff:.2f}%")
print(f"Shape Similarity (Chamfer-based): {chamfer:.2f}%")